<a href="https://colab.research.google.com/github/VladislavKlekovkin/s1mulation/blob/master/1_cross_entropy_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You need to run this code from the google colab


In [1]:
import os
import sys
!git clone https://github.com/VladislavKlekovkin/s1mulation.git 
sys.path.append(os.path.join('./', 's1mulation'))

Cloning into 's1mulation'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 24 (delta 8), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [2]:
from environment import Space

#import gym
from collections import namedtuple
import numpy as np
from torch.utils.tensorboard import SummaryWriter


import torch
import torch.nn as nn
from torch import optim


HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [4]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [5]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [6]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [7]:
env = Space()
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.width * env.height
n_actions = len(env.action_space)

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break
writer.close()

Evn initializing
0: loss=1.386, reward_mean=1.0, reward_bound=1.0
1: loss=1.386, reward_mean=1.0, reward_bound=1.0
2: loss=1.383, reward_mean=1.0, reward_bound=1.0
3: loss=1.382, reward_mean=1.0, reward_bound=1.0
4: loss=1.375, reward_mean=1.0, reward_bound=1.0
5: loss=1.371, reward_mean=1.0, reward_bound=1.0
6: loss=1.361, reward_mean=1.0, reward_bound=1.0
7: loss=1.362, reward_mean=1.0, reward_bound=1.0
8: loss=1.350, reward_mean=1.0, reward_bound=1.0
9: loss=1.358, reward_mean=1.0, reward_bound=1.0
10: loss=1.358, reward_mean=1.0, reward_bound=1.0
11: loss=1.349, reward_mean=1.0, reward_bound=1.0
12: loss=1.353, reward_mean=1.0, reward_bound=1.0
13: loss=1.361, reward_mean=1.0, reward_bound=1.0
14: loss=1.358, reward_mean=1.0, reward_bound=1.0
15: loss=1.360, reward_mean=1.0, reward_bound=1.0
16: loss=1.352, reward_mean=1.0, reward_bound=1.0
17: loss=1.359, reward_mean=1.0, reward_bound=1.0
18: loss=1.353, reward_mean=1.0, reward_bound=1.0
19: loss=1.357, reward_mean=1.0, reward_bou

KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./

'__main__'

In [9]:
state

NameError: ignored